In [30]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
DRIVE_ROOT = "/content/drive/MyDrive/BTP_colab"
import os
for d in ["models","logs","reports","reports/figures"]:
    os.makedirs(f"{DRIVE_ROOT}/{d}", exist_ok=True)
print("Drive outputs ->", DRIVE_ROOT)


Mounted at /content/drive
Drive outputs -> /content/drive/MyDrive/BTP_colab


In [31]:
%cd /content
!rm -rf BTP
!mkdir -p BTP/src/data BTP/src/models BTP/src/xai BTP/models BTP/logs BTP/reports/figures BTP/data/raw BTP/data/processed
%cd /content/BTP
!touch src/__init__.py src/data/__init__.py src/models/__init__.py src/xai/__init__.py
import os
print("Project root:", os.getcwd())


/content
/content/BTP
Project root: /content/BTP


In [32]:
%%writefile config.yaml
data:
  raw_dir: "data/raw"
  processed_dir: "data/processed"
  img_size: 224
  classes: ["glioma","meningioma","pituitary","no_tumor"]

train:
  batch_size: 32
  epochs: 14          # total epochs
  warmup_epochs: 4    # head-only training
  learning_rate: 0.0001
  finetune_lr: 0.00001
  model_name: "resnet50"
  freeze_base: true
  unfreeze_block: "layer4"   # options: layer4 or all
  max_images: 0              # 0 or null = use all images

callbacks:
  early_stopping: true
  patience: 5

paths:
  models_dir: "/content/drive/MyDrive/BTP_colab/models"
  logs_dir: "/content/drive/MyDrive/BTP_colab/logs"
  reports_dir: "/content/drive/MyDrive/BTP_colab/reports"


Writing config.yaml


In [33]:
!pip -q install albumentations seaborn pyyaml grad-cam
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"


In [34]:
from google.colab import files
print("Click 'Choose files' and select kaggle.json (Kaggle Account > Create New API Token)")
_ = files.upload()

!pip -q install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

%cd /content/BTP/data/raw
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset -q
!unzip -q brain-tumor-mri-dataset.zip

# Move Training/Testing up if nested
import os, glob, shutil
for name in ['Training','Testing']:
    hits = [p for p in glob.glob('**/'+name, recursive=True) if os.path.isdir(p)]
    if hits and not os.path.isdir(name):
        shutil.move(hits[0], name)
print("Has Training:", os.path.isdir("Training"), "Has Testing:", os.path.isdir("Testing"))
%cd /content/BTP


Click 'Choose files' and select kaggle.json (Kaggle Account > Create New API Token)


Saving kaggle.json to kaggle.json
/content/BTP/data/raw
Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
License(s): CC0-1.0
Has Training: True Has Testing: True
/content/BTP


In [35]:
%%writefile src/data/generator.py
import os, yaml
from tensorflow.keras.preprocessing.image import ImageDataGenerator
cfg = yaml.safe_load(open("config.yaml"))
IMG = cfg['data']['img_size']
def get_generators(batch_size):
    proc = cfg['data']['processed_dir']
    train_datagen = ImageDataGenerator(rescale=1./255,
                                       rotation_range=15,
                                       width_shift_range=0.1,
                                       height_shift_range=0.1,
                                       zoom_range=0.1,
                                       horizontal_flip=True,
                                       fill_mode='nearest')
    test_datagen = ImageDataGenerator(rescale=1./255)
    train_gen = train_datagen.flow_from_directory(os.path.join(proc,'train'),
                                                  target_size=(IMG,IMG),
                                                  batch_size=batch_size,
                                                  class_mode='categorical', shuffle=True)
    val_gen = test_datagen.flow_from_directory(os.path.join(proc,'val'),
                                               target_size=(IMG,IMG),
                                               batch_size=batch_size,
                                               class_mode='categorical', shuffle=False)
    test_gen = test_datagen.flow_from_directory(os.path.join(proc,'test'),
                                                target_size=(IMG,IMG),
                                                batch_size=1,
                                                class_mode='categorical', shuffle=False)
    return train_gen, val_gen, test_gen


Writing src/data/generator.py


In [36]:
%%writefile src/models/build_model.py
import yaml, tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50, EfficientNetB0
cfg = yaml.safe_load(open("config.yaml"))
def build_model():
    img_size = cfg['data']['img_size']
    n_classes = len(cfg['data']['classes'])
    backbone = cfg['train']['model_name']
    freeze_base = cfg['train'].get('freeze_base', True)
    if backbone.lower() == 'resnet50':
        base = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size,img_size,3))
        last_conv = 'conv5_block3_out'
    elif backbone.lower().startswith('efficient'):
        base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size,img_size,3))
        last_conv = 'top_activation'
    else:
        raise ValueError("Unknown backbone")
    if freeze_base:
        for layer in base.layers: layer.trainable = False
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    out = Dense(n_classes, activation='softmax')(x)
    model = Model(inputs=base.input, outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(cfg['train']['learning_rate']),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model, last_conv


Writing src/models/build_model.py


In [37]:
%%writefile src/utils.py
import os, yaml, matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np, seaborn as sns
cfg = yaml.safe_load(open("config.yaml"))
def save_plot_history(history, out_path):
    plt.figure(figsize=(8,4))
    plt.plot(history.history['accuracy'], label='train_acc')
    plt.plot(history.history['val_accuracy'], label='val_acc')
    plt.title('Accuracy'); plt.legend(); plt.savefig(out_path+"_acc.png"); plt.close()
    plt.figure(figsize=(8,4))
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.title('Loss'); plt.legend(); plt.savefig(out_path+"_loss.png"); plt.close()
def save_confusion(y_true, y_pred, labels, out_path):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels, cmap='Blues')
    plt.ylabel('True'); plt.xlabel('Predicted'); plt.title('Confusion Matrix')
    plt.savefig(out_path); plt.close()
def ensure_reports_dirs():
    os.makedirs(cfg['paths']['reports_dir'], exist_ok=True)
    os.makedirs(os.path.join(cfg['paths']['reports_dir'],'figures'), exist_ok=True)


Writing src/utils.py


In [38]:
%%writefile src/train.py
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import yaml, tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from src.data.generator import get_generators
from src.models.build_model import build_model
from src.utils import save_plot_history, ensure_reports_dirs

cfg = yaml.safe_load(open("config.yaml"))

def unfreeze_top(model, which="layer4"):
    if which == "all":
        for l in model.layers:
            l.trainable = True
    else:
        # Unfreeze only top ResNet block if available
        for l in model.layers:
            if "conv5_block" in l.name or "post_relu" in l.name:
                l.trainable = True

def compile_with_lr(model, lr):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=float(lr)),
                  loss='categorical_crossentropy', metrics=['accuracy'])

def main():
    ensure_reports_dirs()
    batch = cfg['train']['batch_size']
    epochs = int(cfg['train']['epochs'])
    warmup = int(cfg['train']['warmup_epochs'])
    model_dir = cfg['paths']['models_dir']
    os.makedirs(model_dir, exist_ok=True)

    train_gen, val_gen, _ = get_generators(batch)
    model, last_conv = build_model()

    # Phase 1: head-only training
    compile_with_lr(model, cfg['train']['learning_rate'])
    cb = [
        ModelCheckpoint(os.path.join(model_dir,'best_model.h5'), save_best_only=True, monitor='val_loss', verbose=1),
        EarlyStopping(patience=cfg['callbacks']['patience'], restore_best_weights=True),
        ReduceLROnPlateau(patience=3, factor=0.5, verbose=1),
        CSVLogger(os.path.join(cfg['paths']['logs_dir'],'training_log.csv'), append=True)
    ]
    if warmup > 0:
        print(f"Warmup (frozen base) for {warmup} epochs...")
        model.fit(train_gen, validation_data=val_gen, epochs=warmup, callbacks=cb)

    # Phase 2: fine-tune top block (or all) with smaller LR
    unfreeze_top(model, cfg['train'].get('unfreeze_block', 'layer4'))
    compile_with_lr(model, cfg['train']['finetune_lr'])
    remaining = max(0, epochs - warmup)
    if remaining > 0:
        print(f"Fine-tuning for {remaining} epochs...")
        model.fit(train_gen, validation_data=val_gen, epochs=remaining, callbacks=cb)

    save_plot_history(model.history if hasattr(model, "history") else None,
                      os.path.join(cfg['paths']['reports_dir'],'train_val'))
    print("Saved:", os.path.join(model_dir,'best_model.h5'))

if __name__ == "__main__":
    main()


Writing src/train.py


In [39]:
%%writefile src/data/preprocess.py
import os, yaml, cv2, random
from glob import glob
from sklearn.model_selection import train_test_split
cfg = yaml.safe_load(open("config.yaml"))
RAW = cfg['data']['raw_dir']; PROC = cfg['data']['processed_dir']
IMG = cfg['data']['img_size']; CLASSES = cfg['data']['classes']
MAX_IMAGES = int(cfg['train'].get('max_images', 0) or 0)
def ensure_dirs():
    for split in ['train','val','test']:
        for c in CLASSES:
            os.makedirs(os.path.join(PROC, split, c), exist_ok=True)
def files_under(root, classes):
    paths, labels = [], []
    for c in classes:
        for ext in ("*.jpg","*.jpeg","*.png","*.bmp"):
            for f in glob(os.path.join(root, c, ext)):
                paths.append(f); labels.append(c)
    return paths, labels
def stratified_cap(paths, labels, cap):
    if cap <= 0 or cap >= len(paths): return paths, labels
    by = {}
    for p,l in zip(paths,labels): by.setdefault(l, []).append(p)
    total=len(paths); outp=[]; outl=[]; rem=cap
    for c, arr in by.items():
        random.shuffle(arr); take=round(len(arr)*cap/total); take=min(take,len(arr))
        outp+=arr[:take]; outl+=[c]*take; rem-=take
    if rem>0:
        for c, arr in sorted(by.items(), key=lambda kv: -len(kv[1])):
            extra=[p for p in arr if p not in outp]; need=min(rem,len(extra))
            outp+=extra[:need]; outl+=[c]*need; rem-=need
            if rem<=0: break
    idx=list(range(len(outp))); random.shuffle(idx)
    return [outp[i] for i in idx], [outl[i] for i in idx]
def resize_copy(ps, ls, split):
    for p,lab in zip(ps,ls):
        img=cv2.imread(p);
        if img is None: continue
        import numpy as np
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB); img=cv2.resize(img,(IMG,IMG))
        out=os.path.join(PROC,split,lab,os.path.basename(p))
        cv2.imwrite(out, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
def main():
    random.seed(42); ensure_dirs()
    train_root=os.path.join(RAW,"Training"); test_root=os.path.join(RAW,"Testing")
    if not (os.path.isdir(train_root) and os.path.isdir(test_root)):
        raise RuntimeError(f"Expected {train_root} and {test_root}")
    trp,trl=files_under(train_root,CLASSES); tep,tel=files_under(test_root,CLASSES)
    allp=trp+tep; alll=trl+tel
    if MAX_IMAGES and len(allp)>MAX_IMAGES:
        allp, alll = stratified_cap(allp, alll, MAX_IMAGES)
    ptv, pte, ltv, lte = train_test_split(allp, alll, test_size=0.10, stratify=alll, random_state=42)
    ptr, pva, ltr, lva = train_test_split(ptv, ltv, test_size=0.1111, stratify=ltv, random_state=42)
    resize_copy(ptr,ltr,"train"); resize_copy(pva,lva,"val"); resize_copy(pte,lte,"test")
    print(f"Processed -> {PROC} (cap={MAX_IMAGES})")
if __name__=="__main__": main()


Writing src/data/preprocess.py


In [40]:
!python -m src.data.preprocess

import os, sys, importlib
%cd /content/BTP
if "/content/BTP" not in sys.path:
    sys.path.insert(0, "/content/BTP")
print("Has src?", os.path.isdir("src"))
print("Has generator.py?", os.path.isfile("src/data/generator.py"))
importlib.import_module("src.data.generator")
print("Import OK")

!python -m src.train


Processed -> data/processed (cap=0)
/content/BTP
Has src? True
Has generator.py? True
Import OK
2025-09-21 10:45:05.353746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758451505.385981   11748 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758451505.395489   11748 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758451505.456549   11748 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758451505.456597   11748 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more th

In [41]:
%%writefile src/evaluate.py
import yaml, os, numpy as np
from tensorflow.keras.models import load_model
from src.data.generator import get_generators
from src.utils import save_confusion, ensure_reports_dirs
from sklearn.metrics import classification_report

cfg = yaml.safe_load(open("config.yaml"))

def main():
    ensure_reports_dirs()
    _, _, test_gen = get_generators(cfg['train']['batch_size'])
    model = load_model(os.path.join(cfg['paths']['models_dir'],'best_model.h5'))
    preds = model.predict(test_gen, verbose=1)
    y_pred = preds.argmax(axis=1)
    y_true = test_gen.classes
    idx_to_name = {v:k for k,v in test_gen.class_indices.items()}
    labels = sorted(test_gen.class_indices.values())
    target_names = [idx_to_name[i] for i in labels]
    report = classification_report(y_true, y_pred, labels=labels, target_names=target_names, zero_division=0)
    print(report)
    with open(os.path.join(cfg['paths']['reports_dir'],'classification_report.txt'),'w') as f:
        f.write(report)
    save_confusion(y_true, y_pred, target_names, os.path.join(cfg['paths']['reports_dir'],'figures','confusion_matrix.png'))
    print("Reports saved to", cfg['paths']['reports_dir'])

if __name__ == "__main__":
    main()


Writing src/evaluate.py


In [42]:
!python -m src.evaluate


2025-09-21 11:00:59.438691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758452459.470864   16124 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758452459.480619   16124 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758452459.503918   16124 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758452459.503950   16124 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758452459.503955   16124 computation_placer.cc:177] computation placer alr

In [43]:
%%writefile src/gradcam.py
import numpy as np, tensorflow as tf, cv2, yaml
cfg = yaml.safe_load(open("config.yaml"))
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model([model.inputs],[model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None: pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0].numpy()
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]): conv_outputs[..., i] *= pooled_grads[i]
    heatmap = np.maximum(np.mean(conv_outputs, axis=-1), 0)
    heatmap /= (np.max(heatmap)+1e-8)
    return heatmap
def overlay_heatmap(original_img, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    hmap = cv2.resize(heatmap, (original_img.shape[1], original_img.shape[0]))
    hmap = np.uint8(255 * hmap); hmap = cv2.applyColorMap(hmap, colormap)
    overlay = cv2.addWeighted(cv2.cvtColor(original_img, cv2.COLOR_RGB2BGR), 1-alpha, hmap, alpha, 0)
    return cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)


Writing src/gradcam.py


In [44]:
%%writefile src/predict.py
import sys, yaml, cv2, numpy as np, os
from tensorflow.keras.models import load_model
from src.gradcam import make_gradcam_heatmap, overlay_heatmap
cfg = yaml.safe_load(open("config.yaml"))
MODEL_PATH = os.path.join(cfg['paths']['models_dir'],'best_model.h5')
def load_and_preprocess(path):
    img = cv2.imread(path)
    if img is None: raise FileNotFoundError(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    sz = cfg['data']['img_size']
    img_res = cv2.resize(img, (sz, sz))
    img_norm = img_res.astype('float32')/255.0
    return img_res, np.expand_dims(img_norm, axis=0)
def predict_and_explain(path):
    img_orig, x = load_and_preprocess(path)
    model = load_model(MODEL_PATH)
    preds = model.predict(x, verbose=0)[0]
    idx = int(np.argmax(preds)); classes = cfg['data']['classes']
    last_conv = 'conv5_block3_out' if cfg['train']['model_name']=='resnet50' else 'top_activation'
    heatmap = make_gradcam_heatmap(x, model, last_conv, pred_index=idx)
    overlay = overlay_heatmap(img_orig, heatmap)
    outdir = os.path.join(cfg['paths']['reports_dir'],'figures'); os.makedirs(outdir, exist_ok=True)
    fname = os.path.join(outdir, f"gradcam_{classes[idx]}.png")
    cv2.imwrite(fname, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
    print({"predicted": classes[idx], "confidence": float(preds[idx]), "saved": fname})
if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: !python -m src.predict path/to/image.jpg"); sys.exit(1)
    predict_and_explain(sys.argv[1])


Writing src/predict.py


In [45]:
import glob
test_imgs = glob.glob('data/processed/test/*/*.jpg')
print("Test images:", len(test_imgs))
if test_imgs:
    demo = test_imgs[0]
    !python -m src.predict "{demo}"


Test images: 503
2025-09-21 11:02:02.400864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758452522.434754   16490 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758452522.444489   16490 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758452522.467793   16490 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758452522.467824   16490 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758452522.467832   16490 computation_placer.cc:177] compu

In [46]:
!ls -lah /content/drive/MyDrive/BTP_colab/models
!ls -lah /content/drive/MyDrive/BTP_colab/reports
!ls -lah /content/drive/MyDrive/BTP_colab/reports/figures


total 211M
-rw------- 1 root root 211M Sep 21 10:59 best_model.h5
total 50K
-rw------- 1 root root  434 Sep 21 11:02 classification_report.txt
drwx------ 2 root root 4.0K Sep 21 11:02 figures
-rw------- 1 root root  23K Sep 21 11:00 train_val_acc.png
-rw------- 1 root root  23K Sep 21 11:00 train_val_loss.png
total 92K
-rw------- 1 root root 28K Sep 21 11:02 confusion_matrix.png
-rw------- 1 root root 65K Sep 21 11:02 gradcam_glioma.png


In [47]:
import os, numpy as np, glob, cv2, yaml
from tensorflow.keras.models import load_model
from src.data.generator import get_generators
cfg = yaml.safe_load(open("config.yaml"))
model = load_model(os.path.join(cfg['paths']['models_dir'],'best_model.h5'))
_, _, test_gen = get_generators(16)
probs = model.predict(test_gen, verbose=0)
pred = probs.argmax(1); true = test_gen.classes
idx_to_name = {v:k for k,v in test_gen.class_indices.items()}
mis_idx = np.where(pred != true)[0][:36]
outdir = os.path.join(cfg['paths']['reports_dir'],'figures'); os.makedirs(outdir, exist_ok=True)
# Save small grid of misclassifications
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))
for i,k in enumerate(mis_idx[:36]):
    img_path = test_gen.filepaths[k]
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    plt.subplot(6,6,i+1); plt.imshow(img); plt.axis('off')
    plt.title(f"T:{idx_to_name[true[k]]}\nP:{idx_to_name[pred[k]]}", fontsize=8)
plt.tight_layout(); plt.savefig(os.path.join(outdir,"misclassified_grid.png")); plt.close()
print("Saved misclassified_grid.png")


Found 4017 images belonging to 4 classes.
Found 503 images belonging to 4 classes.
Found 503 images belonging to 4 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Saved misclassified_grid.png


In [48]:
print(open("/content/drive/MyDrive/BTP_colab/reports/classification_report.txt").read())


              precision    recall  f1-score   support

      glioma       0.83      0.78      0.81       162
  meningioma       0.81      0.75      0.78       165
    no_tumor       0.00      0.00      0.00         0
   pituitary       0.85      0.97      0.91       176

    accuracy                           0.83       503
   macro avg       0.62      0.62      0.62       503
weighted avg       0.83      0.83      0.83       503



In [49]:
import glob, os
test_imgs = []
for c in ["glioma","meningioma","pituitary","no_tumor"]:
    test_imgs += glob.glob(f"data/processed/test/{c}/*.jpg")[:1]
for p in test_imgs[:4]:
    !python -m src.predict "{p}"


2025-09-21 11:04:38.096142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758452678.118030   17303 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758452678.124308   17303 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758452678.139646   17303 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758452678.139673   17303 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758452678.139676   17303 computation_placer.cc:177] computation placer alr

In [50]:
readme = f"""
# Brain Tumor Detection (ResNet50 + Grad-CAM)

- Dataset: 4-class T1-CE Brain MRI (Glioma/Meningioma/Pituitary/No Tumor)
- Image size: {yaml.safe_load(open('config.yaml'))['data']['img_size']}
- Training: Transfer learning + fine-tuning (see training_log.csv)
- Checkpoint: best_model.h5 (saved to Drive)
- Metrics: See classification_report.txt, confusion_matrix.png
- Explainability: See Grad-CAM overlays in reports/figures

Run order (Colab):
1) Preprocess: python -m src.data.preprocess
2) Train: python -m src.train
3) Evaluate: python -m src.evaluate
4) Grad-CAM demo: python -m src.predict data/processed/test/<class>/<image>.jpg
"""
open("/content/drive/MyDrive/BTP_colab/README_colab.txt","w").write(readme)
print("Wrote README_colab.txt to Drive.")


Wrote README_colab.txt to Drive.


In [53]:
!zip -qr /content/BTP_colab_export.zip /content/drive/MyDrive/BTP_colab

# Download to local machine via browser
from google.colab import files
files.download('/content/BTP_colab_export.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>